## Caltech 101 Preprocessing

A notebook to precompute all of the TSP tours on the caltech-101 dataset

In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

import glob
import skimage
import skimage.io
from skimage.transform import rescale
import scipy.io as sio
import os
from multiprocessing import Pool

import sys
sys.path.append("../src")
from imtools import *
from tsp import *
from curvature import *

In [ ]:
filepaths = glob.glob("../data/caltech-101/*/*.jpg")
print(len(filepaths))

In [ ]:
# Image parameters
thresh = 0.1
p = 1
edge_weight = 10
canny_sigma = 3
density_fac=1
target_points = 2000
n_threads = 10

plt.figure(figsize=(10, 10))

def process_file(f):
    fmat = f[0:-4] + ".mat"
    if not os.path.exists(fmat):
        print(f)
        I = skimage.io.imread(f)
        X = voronoi_stipple(I, thresh, target_points=target_points, p=p, canny_sigma=canny_sigma, edge_weight=edge_weight, n_iters=10, do_plot=False)
        if X.size > 0:
            X = get_tsp_tour(X)
            plt.clf()
            plt.plot(X[:, 0], X[:, 1])
            plt.scatter(X[:, 0], X[:, 1])
            plt.savefig(f[0:-4] + "_tsp.png")
            sio.savemat(fmat, {"X":X})

parpool = Pool(n_threads)
parpool.map(process_file, filepaths)